# Setup

**Note to Mac users:**
* You might get this error: `Unable to revert mtime: /Library/Fonts`
* The solution is to install the *libmagic* brew: `brew install libmagic`
* Reference: [Python MacOS Error: Unable to revert mtime: /Library/Fonts](https://stackoverflow.com/questions/62279920/python-macos-error-unable-to-revert-mtime-library-fonts)

In [ ]:
from setup import *

## Set Variables

Gather the following variables:

* database name
* path to the Net-Manage repository
* private data directory
* path to store the output

**Note:** The inventory file is statically set to *private_data_dir/inventory/hosts*

## Explanation of Variables

### database name

defaults to the current date, in YYYY-MM-DD format

### path to the Net-Manage repository

defaults to current folder

### Path to the Private Data Directory

This variable requires explanation. It is the folder that *contains* the inventory folder. **Do not store that information in Net-Manage directory, because it could be overwritten the next time you do a "git pull"**

### path to store the output

path to store the database and Excel files

defaults to 'private_data_dir/output'

**note:** The Ansible-runner output, like "artifacts", will be stored in the private data directory

#### Example folder structure

* Net-Manage path: *~/source/repos/InsightSSG/Net-Manage*

* Private data directory: *~/tenant_data/*

* Inventory file: *~/tenant_data/inventory/hosts*

In [ ]:
api_key, db, db_path, inventories, nm_path, out_path, private_data_dir = hp.set_vars()
play_path = f'{nm_path}/playbooks'
username, password = hp.get_creds()

# Select and Run Collectors

## Select the hostgroups to run the collectors on

In [ ]:
hostgroup_select = select_hostgroups(private_data_dir)
for key in sorted(hostgroup_select.keys()):
    print(key.upper())
    display(widgets.GridBox(hostgroup_select[key],
                            layout=widgets.Layout(grid_template_columns='repeat(3, 200px)')))
    print('\n')

## Select the collectors to run

In [ ]:
collector_select = select_collectors(collector_select, hostgroup_select)
for key in sorted(collector_select.keys()):
    print(key.upper())
    display(widgets.GridBox(collector_select[key],
                            layout=widgets.Layout(grid_template_columns='repeat(3, 250px)')))
    print('\n')

### Select Meraki Organizations and Networks (if applicable)

**Important! (Please Read):**

* You do not have to enter an organization ID for any of the 'meraki_get_org' functions.
    * If you do not enter any organization IDs, then the script will query all orgs that have API access enabled.

* You do not *have* to enter a network ID for 'meraki_get_network' functions
    * **I strongly recommend you always enter one or more network IDs when running 'meraki_get_network' functions!**
    * If you do not enter any network IDs, then the script will query all network IDs for the orgs you specify.
    * If you do not enter any org IDs or network IDs, the script will query all devices in all orgs. **This will be slow!**
    * If you want to quickly find the network ID for the device you're interested in, then run 'meraki_get_org_devices' instead
        * 'meraki_get_org_devices' does not return as much device-specific data, and executes in seconds, even in orgs with thousands of devices

* Currently there is a bug that can cause the collectors to fail when the following conditions are met:
    1. You use an API key to run collectors on orgs that API key has access to
    1. You use the same database to run collectors on orgs that a *different* API key has access to
    1. This is on my ToDo list to fix. For now, the workaround is to use a separate database.


In [ ]:
import collectors as cl
import helpers as hp
import run_collectors as rc

importlib.reload(cl)
importlib.reload(hp)
importlib.reload(rc)

networks, orgs = hp.get_user_meraki_input()

## Run Collectors

In [ ]:
import collectors as cl
import helpers as hp
import run_collectors as rc

importlib.reload(cl)
importlib.reload(hp)
importlib.reload(rc)

# Create a dataframe of collectors to run
print('COLLECTORS TO RUN\n')
df_collectors = create_collectors_df(collector_select, hostgroup_select)
display(df_collectors)

# Set the timestamp so it will be consistent for all collectors
ts = dt.datetime.now()
ts = ts.strftime('%Y-%m-%d_%H%M')

# Execute the collectors
for idx, row in df_collectors.iterrows():
    ansible_os = row['ansible_os']
    hostgroup = row['hostgroup']
    collector = row['collector']
    result = rc.collect(collector,
                        nm_path,
                        private_data_dir,
                        ts,
                        ansible_os=ansible_os,
                        username=username,
                        password=password,
                        api_key=api_key,
                        hostgroup=hostgroup,
                        play_path=play_path,
                        db_path=db_path,
                        orgs=orgs,
                        networks=networks)
    print(f'RESULT: {ansible_os.upper()} {collector.upper()} COLLECTOR\n')
    display(result)

# Run Validators

In [ ]:
import collectors as cl
import helpers as hp
import run_collectors as rc
import validators as vl

importlib.reload(cl)
importlib.reload(hp)
importlib.reload(rc)
importlib.reload(vl)

In [ ]:
df_diff = vl.f5_vip_availability(db_path, 'f5_vip_availability')
display(df_diff)

In [ ]:
df_diff = vl.f5_pool_availability(db_path, 'f5_pool_availability')
display(df_diff)

In [ ]:
df_diff = vl.f5_pool_member_availability(db_path, 'f5_pool_member_availability')
display(df_diff)